In [1]:
import cv2
import numpy as np
import requests
from tensorflow.keras.models import load_model
import time
from tensorflow.keras.preprocessing import image

In [2]:
model = load_model("fusion_model.keras")

In [3]:
import cv2
import numpy as np

def calculate_blurriness(image):
    """Return the variance of the Laplacian (blurriness score)"""
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    return cv2.Laplacian(gray, cv2.CV_64F).var()

def get_least_blurry_image(image_buffer):
    """Return the image with the highest sharpness (least blurry)"""
    sharpest_image = None
    max_score = -1

    for i, img in enumerate(image_buffer):
        score = calculate_blurriness(img)
        print(f"Image {i} - Sharpness Score: {score}")
        if score > max_score:
            max_score = score
            sharpest_image = img

    return sharpest_image


In [ ]:
#Direction following
direction = {
    0:'Left',
    1:'North',
    2:'Right',
    3:'South'
}


#Server end point for message communication
host_url = 'http://10.10.10.10:5002/update'  

#Server endpoint for video communication
raspberry_pi_ip = 'http://10.10.10.10:5002/video'

#Instanciate the video capture
cap = cv2.VideoCapture(raspberry_pi_ip)

#Avoid latency and using old frames
# cap.set(cv2.CAP_PROP_BUFFERSIZE, 1)


a=0
print('Start')
while a<2:
    # Flush old frames
    for _ in range(50):
        cap.read()

    buffer = []
    while len(buffer) < 10:
        ret, frame = cap.read()
        if ret:
            buffer.append(frame)

    if len(buffer) < 5:
        print("Not enough good frames.")
        continue

    sharpest_frame = get_least_blurry_image(buffer)
    cv2.imshow("Live feed", sharpest_frame)

    resized = cv2.resize(sharpest_frame, (150, 150))
    img_array = np.expand_dims(resized, axis=0)
    
    start_time = time.time()
    prediction = model.predict([img_array, img_array])
    end_time = time.time()

    predicted_class = np.argmax(prediction[0])
    print(f"Predicted Direction: {direction[predicted_class]} (Inference Time: {end_time - start_time:.2f}s)")

    response = requests.post(host_url, json={'message': str(predicted_class)})
    print("Message sent" if response.ok else f"Error: {response.status_code}")

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    a = a+1



Start
Image 0 - Sharpness Score: 462.34660227075153
Image 1 - Sharpness Score: 462.9179787862142
Image 2 - Sharpness Score: 461.96207610978036
Image 3 - Sharpness Score: 460.6861707598263
Image 4 - Sharpness Score: 461.10850077158625
Image 5 - Sharpness Score: 460.2108899855614
Image 6 - Sharpness Score: 460.52292727593294
Image 7 - Sharpness Score: 462.3563702832435
Image 8 - Sharpness Score: 459.91651964941025
Image 9 - Sharpness Score: 459.9908216037327
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 400ms/step
Predicted Direction: North (Inference Time: 0.51s)
Message sent
Image 0 - Sharpness Score: 394.96444719567836
Image 1 - Sharpness Score: 396.51751655272375
Image 2 - Sharpness Score: 394.27078118065737
Image 3 - Sharpness Score: 392.67010852796767
Image 4 - Sharpness Score: 391.97677641292154
Image 5 - Sharpness Score: 391.9433976279257
Image 6 - Sharpness Score: 390.5443409010145
Image 7 - Sharpness Score: 389.3371393101904
Image 8 - Sharpness Score: 389.35994021771756
Image 9 - Sharpness Score

-0.2531923209086503
